In [1]:
%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 744.7 kB/s eta 0:56:11
     ---------------------------------------- 0.0/2.5 GB 824.4 kB/s eta 0:50:45
     ---------------------------------------- 0.0/2.5 GB 849.0 kB/s eta 0:49:16
     ---------------------------------------- 0.0/2.5 GB 855.6 kB/s eta 0:48:53
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 924.4 kB/s eta 0:45:14
     ---

In [2]:
import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric

Installing packages for torch-2.5.1+cu124...
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
     ------------ --------------------------- 0.5/1.7 MB 2.1 MB/s eta 0:00:01
     ------------------ --------------------- 0.8/1.7 MB 1.1 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.5 MB 2.8 MB/s eta 0:00:02
     ------------ --------------------------- 1.0/3.5 MB 2.5 MB/s eta 0:00:01
     ------------------ --------------------- 1.6/3.5 MB 2.5 MB/s eta 0:00:01
     --------------------- ------------------ 1.8/3.5 MB 2.6 MB/s eta 0:00:01
     --------------------------- ------------ 2.4/3.

In [4]:
%pip install scikit-learn imbalanced-learn xgboost

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   -------- ----------------

In [6]:
%pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 1.3 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Hyperparameter Optimization Citeseer
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 17, 23),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.2, 0.8),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 0.9),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=250)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 09:31:01,927] A new study created in memory with name: no-name-8077ec32-fc0c-452a-b52a-53fd337fb55b


Running experiment with seed=1:
 - alpha=0.4818030307513619, K=23, layers=1, hidden=256, out=256
 - norm=layernorm, dropout=0.4096380127607656, margin=0.6298448454130761, lpl_weight=0.42524365771893835
 - ratio=0.13579701531202024, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.6482, LPL: 1.3863, Contrastive: 0.1021
Epoch 50, Loss: 0.6097, LPL: 1.3863, Contrastive: 0.0351
Epoch 100, Loss: 0.6095, LPL: 1.3863, Contrastive: 0.0348
Epoch 150, Loss: 0.6095, LPL: 1.3863, Contrastive: 0.0347
Epoch 200, Loss: 0.6094, LPL: 1.3863, Contrastive: 0.0347
 - Metrics: Accuracy=0.9110, F1=0.7967, Recall=0.8274, Precision=0.7682
Running experiment with seed=2:
 - alpha=0.4818030307513619, K=23, layers=1, hidden=256, out=256
 - norm=layernorm, dropout=0.4096380127607656, margin=0.6298448454130761, lpl_weight=0.42524365771893835
 - ratio=0.13579701531202024, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.6482, LPL: 1.3863, Contrastive: 0.1021
Epoch 50, Loss:

[I 2025-02-21 09:34:24,500] Trial 0 finished with value: 0.795054945054945 and parameters: {'alpha': 0.4818030307513619, 'K': 23, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'layernorm', 'dropout': 0.4096380127607656, 'margin': 0.6298448454130761, 'lpl_weight': 0.42524365771893835, 'ratio': 0.13579701531202024, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 0 with value: 0.795054945054945.


 - Metrics: Accuracy=0.9086, F1=0.7912, Recall=0.8217, Precision=0.7629
Done. Results written to citeseer_experimentations\citeseer_scar_2102093101.csv.
Average F1 over 5 seeds: 0.7951 ± 0.0033
Running experiment with seed=1:
 - alpha=0.3397053026424934, K=23, layers=2, hidden=128, out=128
 - norm=None, dropout=0.20337630613836738, margin=0.3522537449966624, lpl_weight=0.557051918594492
 - ratio=0.35611672939063976, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.8416, LPL: 1.3863, Contrastive: 0.1565
Epoch 50, Loss: 0.8202, LPL: 1.3863, Contrastive: 0.1082
Epoch 100, Loss: 0.8196, LPL: 1.3863, Contrastive: 0.1070
Epoch 150, Loss: 0.8195, LPL: 1.3863, Contrastive: 0.1066
 - Metrics: Accuracy=0.7586, F1=0.6196, Recall=0.9330, Precision=0.4638
Running experiment with seed=2:
 - alpha=0.3397053026424934, K=23, layers=2, hidden=128, out=128
 - norm=None, dropout=0.20337630613836738, margin=0.3522537449966624, lpl_weight=0.557051918594492
 - ratio=0.35611672939063976, p

[I 2025-02-21 09:36:34,168] Trial 1 finished with value: 0.6205589767882521 and parameters: {'alpha': 0.3397053026424934, 'K': 23, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'norm': None, 'dropout': 0.20337630613836738, 'margin': 0.3522537449966624, 'lpl_weight': 0.557051918594492, 'ratio': 0.35611672939063976, 'aggregation': 'sum', 'treatment': 'relabeling'}. Best is trial 0 with value: 0.795054945054945.


...

 - Metrics: Accuracy=0.9267, F1=0.8315, Recall=0.8588, Precision=0.8059
Done. Results written to citeseer_experimentations\citeseer_scar_2102104128.csv.
Average F1 over 5 seeds: 0.8343 ± 0.0047
Running experiment with seed=1:
 - alpha=0.8056068614320757, K=17, layers=2, hidden=64, out=128
 - norm=graphnorm, dropout=0.24505980327333826, margin=0.6023138521615269, lpl_weight=0.36346256621785694
 - ratio=0.49914242523070973, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.5976, LPL: 1.3863, Contrastive: 0.1473
Epoch 50, Loss: 0.5306, LPL: 1.3863, Contrastive: 0.0421
Epoch 100, Loss: 0.5300, LPL: 1.3863, Contrastive: 0.0410
Epoch 150, Loss: 0.5298, LPL: 1.3863, Contrastive: 0.0407
 - Metrics: Accuracy=0.8608, F1=0.7362, Recall=0.9215, Precision=0.6129
Running experiment with seed=2:
 - alpha=0.8056068614320757, K=17, layers=2, hidden=64, out=128
 - norm=graphnorm, dropout=0.24505980327333826, margin=0.6023138521615269, lpl_weight=0.36346256621785694
 - ratio=0.49914242523

[I 2025-02-21 10:43:26,423] Trial 48 finished with value: 0.732263541506266 and parameters: {'alpha': 0.8056068614320757, 'K': 17, 'layers': 2, 'hidden_channels': 64, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.24505980327333826, 'margin': 0.6023138521615269, 'lpl_weight': 0.36346256621785694, 'ratio': 0.49914242523070973, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 35 with value: 0.8482742872108142.


 - Metrics: Accuracy=0.8581, F1=0.7309, Recall=0.9144, Precision=0.6087
Done. Results written to citeseer_experimentations\citeseer_scar_2102104204.csv.
Average F1 over 5 seeds: 0.7323 ± 0.0058
Running experiment with seed=1:
 - alpha=0.49424877445323373, K=18, layers=2, hidden=128, out=128
 - norm=graphnorm, dropout=0.21050103064739623, margin=0.5521141684212254, lpl_weight=0.5618379756997395
 - ratio=0.1602350372759239, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.8429, LPL: 1.3863, Contrastive: 0.1462
 - Metrics: Accuracy=0.9291, F1=0.8354, Recall=0.8545, Precision=0.8172
Running experiment with seed=2:
 - alpha=0.49424877445323373, K=18, layers=2, hidden=128, out=128
 - norm=graphnorm, dropout=0.21050103064739623, margin=0.5521141684212254, lpl_weight=0.5618379756997395
 - ratio=0.1602350372759239, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.8429, LPL: 1.3863, Contrastive: 0.1462
 - Metrics: Accuracy=0.9324, F1=0.8441, Recall=0.8688, Preci

[I 2025-02-21 10:44:03,593] Trial 49 finished with value: 0.8387050827853739 and parameters: {'alpha': 0.49424877445323373, 'K': 18, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.21050103064739623, 'margin': 0.5521141684212254, 'lpl_weight': 0.5618379756997395, 'ratio': 0.1602350372759239, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 35 with value: 0.8482742872108142.


 - Metrics: Accuracy=0.9315, F1=0.8419, Recall=0.8659, Precision=0.8192
Done. Results written to citeseer_experimentations\citeseer_scar_2102104326.csv.
Average F1 over 5 seeds: 0.8387 ± 0.0039
Best trial:
  Average F1: 0.8482742872108142
  Best parameters:
    alpha: 0.8793493737771763
    K: 17
    layers: 2
    hidden_channels: 128
    out_channels: 128
    norm: graphnorm
    dropout: 0.23775361637105552
    margin: 0.5195497901567272
    lpl_weight: 0.5548792276938219
    ratio: 0.13442050105930825
    aggregation: sum
    treatment: removal


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 15, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=250)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 11:03:38,552] A new study created in memory with name: no-name-51768b11-9e2d-4414-b07e-f364f0127f2d


Running experiment with seed=1:
 - alpha=0.19318242593081886, K=15, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.42110600860832226, margin=0.9239927738665273, lpl_weight=0.9572821553708265
 - ratio=0.43303873683086513, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.3362, LPL: 1.3863, Contrastive: 0.2137
 - Metrics: Accuracy=0.8383, F1=0.6691, Recall=0.7760, Precision=0.5881
Running experiment with seed=2:
 - alpha=0.19318242593081886, K=15, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.42110600860832226, margin=0.9239927738665273, lpl_weight=0.9572821553708265
 - ratio=0.43303873683086513, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.3362, LPL: 1.3863, Contrastive: 0.2137
 - Metrics: Accuracy=0.8644, F1=0.7111, Recall=0.7917, Precision=0.6453
Running experiment with seed=3:
 - alpha=0.19318242593081886, K=15, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.42110600860832226, margin=0.9239927738665273, lpl_w

[I 2025-02-21 11:04:27,032] Trial 0 finished with value: 0.7005643791833988 and parameters: {'alpha': 0.19318242593081886, 'K': 15, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.42110600860832226, 'margin': 0.9239927738665273, 'lpl_weight': 0.9572821553708265, 'ratio': 0.43303873683086513, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 0 with value: 0.7005643791833988.


 - Metrics: Accuracy=0.8605, F1=0.7048, Recall=0.7903, Precision=0.6361
Done. Results written to citeseer_experimentations\citeseer_sar_2102110338.csv.
Average F1 over 5 seeds: 0.7006 ± 0.0159
Running experiment with seed=1:
 - alpha=0.8736943955842099, K=22, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.1369219263148594, margin=0.5533481495913736, lpl_weight=0.5616376297689698
 - ratio=0.3416643268199105, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.8284, LPL: 1.3863, Contrastive: 0.1137
Epoch 50, Loss: 0.8014, LPL: 1.3863, Contrastive: 0.0520
Epoch 100, Loss: 0.8010, LPL: 1.3863, Contrastive: 0.0511
Epoch 150, Loss: 0.8009, LPL: 1.3863, Contrastive: 0.0510
 - Metrics: Accuracy=0.7680, F1=0.6267, Recall=0.9244, Precision=0.4740
Running experiment with seed=2:
 - alpha=0.8736943955842099, K=22, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.1369219263148594, margin=0.5533481495913736, lpl_weight=0.5616376297689698
 - ratio=0.34166432681

[I 2025-02-21 11:06:36,575] Trial 1 finished with value: 0.62321083172147 and parameters: {'alpha': 0.8736943955842099, 'K': 22, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.1369219263148594, 'margin': 0.5533481495913736, 'lpl_weight': 0.5616376297689698, 'ratio': 0.3416643268199105, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 0 with value: 0.7005643791833988.


...

 - Metrics: Accuracy=0.9375, F1=0.8429, Recall=0.7960, Precision=0.8957
Done. Results written to citeseer_experimentations\citeseer_sar_2102122100.csv.
Average F1 over 5 seeds: 0.8314 ± 0.0085
Running experiment with seed=1:
 - alpha=0.46143822782565064, K=17, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.26755087144223877, margin=0.6422637921530638, lpl_weight=0.6800360078051488
 - ratio=0.11950420241251236, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.9998, LPL: 1.3863, Contrastive: 0.1784
 - Metrics: Accuracy=0.9261, F1=0.8139, Recall=0.7675, Precision=0.8663
Running experiment with seed=2:
 - alpha=0.46143822782565064, K=17, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.26755087144223877, margin=0.6422637921530638, lpl_weight=0.6800360078051488
 - ratio=0.11950420241251236, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.9998, LPL: 1.3863, Contrastive: 0.1784
 - Metrics: Accuracy=0.9291, F1=0.8201, Recall=0.7675, Prec

[I 2025-02-21 12:22:22,741] Trial 48 finished with value: 0.8243460344734677 and parameters: {'alpha': 0.46143822782565064, 'K': 17, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'layernorm', 'dropout': 0.26755087144223877, 'margin': 0.6422637921530638, 'lpl_weight': 0.6800360078051488, 'ratio': 0.11950420241251236, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 38 with value: 0.8455717941380525.


 - Metrics: Accuracy=0.9354, F1=0.8355, Recall=0.7789, Precision=0.9010
Done. Results written to citeseer_experimentations\citeseer_sar_2102122141.csv.
Average F1 over 5 seeds: 0.8243 ± 0.0073
Running experiment with seed=1:
 - alpha=0.5737913294462824, K=18, layers=3, hidden=256, out=128
 - norm=layernorm, dropout=0.2658876115886441, margin=0.7344059696913318, lpl_weight=0.7186557437315787
 - ratio=0.1466624311788599, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0325, LPL: 1.3863, Contrastive: 0.1288
 - Metrics: Accuracy=0.9068, F1=0.7693, Recall=0.7375, Precision=0.8040
Running experiment with seed=2:
 - alpha=0.5737913294462824, K=18, layers=3, hidden=256, out=128
 - norm=layernorm, dropout=0.2658876115886441, margin=0.7344059696913318, lpl_weight=0.7186557437315787
 - ratio=0.1466624311788599, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0325, LPL: 1.3863, Contrastive: 0.1288
 - Metrics: Accuracy=0.9092, F1=0.7726, Recall=0.7318, Precision=

[I 2025-02-21 12:23:05,622] Trial 49 finished with value: 0.7756226033086795 and parameters: {'alpha': 0.5737913294462824, 'K': 18, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'layernorm', 'dropout': 0.2658876115886441, 'margin': 0.7344059696913318, 'lpl_weight': 0.7186557437315787, 'ratio': 0.1466624311788599, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 38 with value: 0.8455717941380525.


 - Metrics: Accuracy=0.9137, F1=0.7817, Recall=0.7332, Precision=0.8371
Done. Results written to citeseer_experimentations\citeseer_sar_2102122222.csv.
Average F1 over 5 seeds: 0.7756 ± 0.0110
Best trial:
  Average F1: 0.8455717941380525
  Best parameters:
    alpha: 0.37723710900430574
    K: 20
    layers: 2
    hidden_channels: 256
    out_channels: 128
    norm: layernorm
    dropout: 0.2653392372389523
    margin: 0.5902726074174123
    lpl_weight: 0.5979423122947367
    ratio: 0.13594889546650762
    aggregation: sum
    treatment: removal


### Hyperparameter Optimization Cora
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 12:23:05,650] A new study created in memory with name: no-name-4468d66a-049b-414c-8ec5-8a1409d48901
Processing...
Done!


Running experiment with seed=1:
 - alpha=0.6590275255796161, K=34, layers=2, hidden=256, out=64
 - norm=graphnorm, dropout=0.252224978557246, margin=0.801479736149222, lpl_weight=0.10820398144332889
 - ratio=0.3781283040605532, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.2991, LPL: 1.3863, Contrastive: 0.1672
 - Metrics: Accuracy=0.7947, F1=0.7347, Recall=0.9413, Precision=0.6025
Running experiment with seed=2:
 - alpha=0.6590275255796161, K=34, layers=2, hidden=256, out=64
 - norm=graphnorm, dropout=0.252224978557246, margin=0.801479736149222, lpl_weight=0.10820398144332889
 - ratio=0.3781283040605532, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.2991, LPL: 1.3863, Contrastive: 0.1672
 - Metrics: Accuracy=0.7969, F1=0.7376, Recall=0.9450, Precision=0.6049
Running experiment with seed=3:
 - alpha=0.6590275255796161, K=34, layers=2, hidden=256, out=64
 - norm=graphnorm, dropout=0.252224978557246, margin=0.801479736149222, lpl_weight=0.108

[I 2025-02-21 12:23:41,175] Trial 0 finished with value: 0.7356870229007634 and parameters: {'alpha': 0.6590275255796161, 'K': 34, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'norm': 'graphnorm', 'dropout': 0.252224978557246, 'margin': 0.801479736149222, 'lpl_weight': 0.10820398144332889, 'ratio': 0.3781283040605532, 'aggregation': 'sum', 'treatment': 'relabeling'}. Best is trial 0 with value: 0.7356870229007634.


 - Metrics: Accuracy=0.7954, F1=0.7357, Recall=0.9425, Precision=0.6033
Done. Results written to cora_experimentations\cora_scar_2102122305.csv.
Average F1 over 5 seeds: 0.7357 ± 0.0029
Running experiment with seed=1:
 - alpha=0.9541489676457723, K=33, layers=3, hidden=128, out=256
 - norm=graphnorm, dropout=0.4793875130120685, margin=0.3521159342119853, lpl_weight=0.9293264066487583
 - ratio=0.23356207667182172, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3006, LPL: 1.3863, Contrastive: 0.1732
Epoch 50, Loss: 1.2961, LPL: 1.3863, Contrastive: 0.1096
Epoch 100, Loss: 1.2960, LPL: 1.3863, Contrastive: 0.1085
 - Metrics: Accuracy=0.9298, F1=0.8872, Recall=0.9132, Precision=0.8626
Running experiment with seed=2:
 - alpha=0.9541489676457723, K=33, layers=3, hidden=128, out=256
 - norm=graphnorm, dropout=0.4793875130120685, margin=0.3521159342119853, lpl_weight=0.9293264066487583
 - ratio=0.23356207667182172, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Lo

[I 2025-02-21 12:24:50,664] Trial 1 finished with value: 0.8753877722852119 and parameters: {'alpha': 0.9541489676457723, 'K': 33, 'layers': 3, 'hidden_channels': 128, 'out_channels': 256, 'norm': 'graphnorm', 'dropout': 0.4793875130120685, 'margin': 0.3521159342119853, 'lpl_weight': 0.9293264066487583, 'ratio': 0.23356207667182172, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 1 with value: 0.8753877722852119.


...

 - Metrics: Accuracy=0.9298, F1=0.8840, Recall=0.8851, Precision=0.8829
Done. Results written to cora_experimentations\cora_scar_2102130215.csv.
Average F1 over 5 seeds: 0.8904 ± 0.0044
Running experiment with seed=1:
 - alpha=0.18801375193474684, K=33, layers=1, hidden=256, out=256
 - norm=graphnorm, dropout=0.40799894600349557, margin=0.5668723673535981, lpl_weight=0.8376606198246364
 - ratio=0.43637318816380954, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.1804, LPL: 1.3863, Contrastive: 0.1177
 - Metrics: Accuracy=0.8711, F1=0.8125, Recall=0.9242, Precision=0.7248
Running experiment with seed=2:
 - alpha=0.18801375193474684, K=33, layers=1, hidden=256, out=256
 - norm=graphnorm, dropout=0.40799894600349557, margin=0.5668723673535981, lpl_weight=0.8376606198246364
 - ratio=0.43637318816380954, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.1804, LPL: 1.3863, Contrastive: 0.1177
 - Metrics: Accuracy=0.8678, F1=0.8106, Recall=0.9364, Precision

[I 2025-02-21 13:04:02,332] Trial 40 finished with value: 0.8103592394893641 and parameters: {'alpha': 0.18801375193474684, 'K': 33, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'graphnorm', 'dropout': 0.40799894600349557, 'margin': 0.5668723673535981, 'lpl_weight': 0.8376606198246364, 'ratio': 0.43637318816380954, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 17 with value: 0.8982590092902513.


 - Metrics: Accuracy=0.8719, F1=0.8147, Recall=0.9328, Precision=0.7232
Done. Results written to cora_experimentations\cora_scar_2102130326.csv.
Average F1 over 5 seeds: 0.8104 ± 0.0031
Running experiment with seed=1:
 - alpha=0.50462741786112, K=30, layers=1, hidden=256, out=128
 - norm=layernorm, dropout=0.49923307597797684, margin=0.49386231901315814, lpl_weight=0.9560666055670407
 - ratio=0.2432765788527112, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3333, LPL: 1.3863, Contrastive: 0.1811
Epoch 50, Loss: 1.3284, LPL: 1.3863, Contrastive: 0.0678
Epoch 100, Loss: 1.3283, LPL: 1.3863, Contrastive: 0.0664
 - Metrics: Accuracy=0.9350, F1=0.8947, Recall=0.9144, Precision=0.8759
Running experiment with seed=2:
 - alpha=0.50462741786112, K=30, layers=1, hidden=256, out=128
 - norm=layernorm, dropout=0.49923307597797684, margin=0.49386231901315814, lpl_weight=0.9560666055670407
 - ratio=0.2432765788527112, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss

[I 2025-02-21 13:05:20,031] Trial 41 finished with value: 0.8953497403916127 and parameters: {'alpha': 0.50462741786112, 'K': 30, 'layers': 1, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'layernorm', 'dropout': 0.49923307597797684, 'margin': 0.49386231901315814, 'lpl_weight': 0.9560666055670407, 'ratio': 0.2432765788527112, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 17 with value: 0.8982590092902513.


 - Metrics: Accuracy=0.9346, F1=0.8950, Recall=0.9218, Precision=0.8697
Done. Results written to cora_experimentations\cora_scar_2102130402.csv.
Average F1 over 5 seeds: 0.8953 ± 0.0029
Running experiment with seed=1:
 - alpha=0.7680936227333292, K=31, layers=1, hidden=256, out=256
 - norm=layernorm, dropout=0.4958929535051545, margin=0.48540601847611264, lpl_weight=0.9455304164114224
 - ratio=0.24624220327993254, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3211, LPL: 1.3863, Contrastive: 0.1895
 - Metrics: Accuracy=0.9276, F1=0.8833, Recall=0.9071, Precision=0.8608
Running experiment with seed=2:
 - alpha=0.7680936227333292, K=31, layers=1, hidden=256, out=256
 - norm=layernorm, dropout=0.4958929535051545, margin=0.48540601847611264, lpl_weight=0.9455304164114224
 - ratio=0.24624220327993254, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3211, LPL: 1.3863, Contrastive: 0.1895


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
